<a href="https://colab.research.google.com/github/jamg-upv/Impacto-citas-articulo/blob/main/impactoencitas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

How to cite this work:

------------

Este codigo permite integrar una tabla con los datos de un conjunto de articulos y sus numeros de citas, normalizarlas teniendo en cuenta las tablas de citas mundiales (scopus) o baseline de (WOS), y añadir las métricas de impacto de la revista en el año de publicación del articulo.
Son necesarias 3 tablas para cada proveedor (scopus y WOS)


Fuente de datos SCOPUS https://elsevier.com/?a=734751


Tareas 
1. Lista de 25 DOIs Banco de Pruebas1  (meritos (5 autores "al azar") )
2. Descargar como cvs bancopruebas1sco tabla Scopus  de 25 meritos (5 autores "al azar") a partir de los DOI
1. Descargar como cvs bancopruebas1wos tabla WOS de los 25 meritos a partir de los DOI
1. preparar un prototipo de la tabla que penamos crear para consultar a compañeros del equipo
2.   



Pasos previos. Librerias y configuración

In [1]:
# Importamos ls librería de Python que necesitaremos en este notebook

import pandas as pd  # Pandas permite manipular tablas 
import matplotlib.pyplot as plt  # Para hacer gráficas
import seaborn as sns  # Para hacer gráficas pero necesita matplotlib y deja unos graficos chulisismos 
sns.set_style('whitegrid')  # Pone una malla de fondo bastante mona para Seaborn

import os #rename files
#import xlwings as xw #permite leer docuemtnos xlb me da error y no lo puedo instalar

# Redondeamos las salidas a dos decimales
%precision %.2f

# from google.colab import drive
# drive.mount('/content/drive')



'%.2f'

Carga de *datos* Indicadores de revistas SCOPUS

In [ ]:
#métricas de revistas Scopus
# si lo descargamos directo de la web Ç(ventaja es que con una descarga siempre está accesible y no hay que volver a descargarlo durante la sesión)
# el archivo de SCopus de 10 años ocupa demasiados GB (60gb) y no se puede alojar en drive (no debería esar en un acceso libre en github porque es propiedad de Scopus)
   #!wget https://elsevier.com/?a=734751 
#es XLB, hay que convertirlo a mano XLS porque xlb requiere de librerias que no tengo instaladas en colab
# y hacerlo accesible aquí
#la primer pesaña son definiciones de indicadores, la segunra el el año más reciente y luego 9 mas con los anteriores. La ultima pestaña es un clasificación de categorias
## limpio a mano las pestañas no usadas (priemera y ultima) y dejo solo los 7 años más recientes (para que sean solo 44mb y me quepa en google drive)
!wget #pendiente completar con una ruta HTTPS la carga del fichero (de momento lo hago manual, que tarda una eternidad cada vez)

# en general funciona mejor con archivos csv, que con excel. tendre que partir el archivo excel en csv

#renbombrar el fichero local descargado para que no sea tan feote
#os.rename(r'/content/edit?usp=sharing',r'/content/scopus.xlsx')

--2022-02-07 19:04:29--  https://docs.google.com/spreadsheets/d/1gY0gVLoYxCN522Z_6WpzggG3IVodJlMBxhE8J_b__mg/edit?usp=sharing
Resolving docs.google.com (docs.google.com)... 142.250.103.100, 142.250.103.102, 142.250.103.139, ...
Connecting to docs.google.com (docs.google.com)|142.250.103.100|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘edit?usp=sharing’

edit?usp=sharing        [ <=>                ] 331.04K  --.-KB/s    in 0.1s    

2022-02-07 19:04:32 (2.53 MB/s) - ‘edit?usp=sharing’ saved [338984]



In [ ]:
# si sibo el fichero a mano a googlecolab (varios minutos en subir y 130seg en cargar el dataframe)

dfs = pd.read_excel('/content/CiteScore-2014-2020-new-methodology-October-2021.xlsx',engine='openpyxl',sheet_name=None)
dfs

In [ ]:
# Otra forma de hacer exactamente lo mismo de arriba (130 segundo)

xl_file = pd.ExcelFile('/content/CiteScore-2014-2020-new-methodology-October-2021.xlsx')

dfs2 = {sheet_name: xl_file.parse(sheet_name) 
          for sheet_name in xl_file.sheet_names}
dfs2

In [ ]:
##pendiente automtizar la creación de un CSV a partir de las pestañas, añadiendo una columna del año (y rotulando citescore solo sin años en las columnas de datos)

Preparacion de las pestañas d la excel
Quitar espacioes en los rotulos del encabezado
Estandarizar los nombres de las columnas

ScopusSourceID	Title	CitationCount	ScholarlyOutput	PercentCited	CiteScore	SNIP	SJR	ScopusASJCCode	ScopusSubSubjectArea	Percentile	RANK	RankOutOf	Publisher	Type	OpenAccess	Quartile	Top10	URLScopusSourceID	PrintISSN	ElectISSN


In [9]:
# importar tablas de metricas revisas SCOPUS (la excel original la he convertido en un CVS por cada pestaña)
# hay que importar los csv y ñadir a cada uno un campo con el año y luego fusionarlos en una tabla unica o trabajarlos como tablas separadas (no sé que será más eficiente)
# El csv subido a Github no funciona (crea tags de html en el archivo y no se lee luego con pandas)
!wget https://github.com/jamg-upv/Impacto-citas-articulo/blob/main/tablasinput/CiteScore2020.csv


--2022-02-08 12:57:27--  https://github.com/jamg-upv/Impacto-citas-articulo/blob/main/tablasinput/CiteScore2020.csv
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘CiteScore2020.csv’

CiteScore2020.csv       [ <=>                ] 167.57K  --.-KB/s    in 0.01s   

2022-02-08 12:57:27 (14.1 MB/s) - ‘CiteScore2020.csv’ saved [171593]



In [12]:
CS2020= pd.read_csv('/content/CiteScore2020.csv',sep=';')
CS2020

,ScopusSourceID,Title,CitationCount,ScholarlyOutput,PercentCited,CiteScore,SNIP,SJR,ScopusASJCCode,ScopusSubSubjectArea,Percentile,RANK,RankOutOf,Publisher,Type,OpenAccess,Quartile,Top10,URLScopusSourceID,PrintISSN,ElectISSN
0,21101021140,1700-tal: Nordic Journal for Eighteenth-Centur...,0,5,0,0,0,0,1213,Visual Arts and Performing Arts,6,466,532,Swedish Society for Eighteenth-Century Studies,j,YES,4,FALSO,https://www.scopus.com/sourceid/21101021140,NaN,NaN
1,21101021140,1700-tal: Nordic Journal for Eighteenth-Centur...,0,5,0,0,0,0,1202,History,3,1235,1328,Swedish Society for Eighteenth-Century Studies,j,YES,4,FALSO,https://www.scopus.com/sourceid/21101021140,NaN,NaN
2,21101021140,1700-tal: Nordic Journal for Eighteenth-Centur...,0,5,0,0,0,0,3316,Cultural Studies,2,986,1037,Swedish Society for Eighteenth-Century Studies,j,YES,4,FALSO,https://www.scopus.com/sourceid/21101021140,NaN,NaN
3,21100404576,2D Materials,12778,920,90,"13,9","1,402","2,702",2210,Mechanical Engineering,97,14,596,Institute of Physics Publishing,j,NO,1,VERDADERO,https://www.scopus.com/sourceid/21100404576,NaN,20531583
4,21100404576,2D Materials,12778,920,90,"13,9","1,402","2,702",2211,Mechanics of Materials,97,11,377,Institute of Physics Publishing,j,NO,1,VERDADERO,https://www.scopus.com/sourceid/21100404576,NaN,20531583
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59339,28883,Zygon,301,237,56,"1,3","0,853","0,222",3304,Education,46,707,1319,Wiley-Blackwell,j,NO,3,FALSO,https://www.scopus.com/sourceid/28883,5912385,14679744
59340,19073,Zygote,588,206,69,"2,9","0,559","0,416",1309,Developmental Biology,25,61,81,Cambridge University Press,j,NO,3,FALSO,https://www.scopus.com/sourceid/19073,9671994,14698730
59341,19073,Zygote,588,206,69,"2,9","0,559","0,416",1307,Cell Biology,22,218,279,Cambridge University Press,j,NO,4,FALSO,https://www.scopus.com/sourceid/19073,9671994,14698730
59342,19700173167,Zywnosc. Nauka. Technologia. Jakosc/Food. Scie...,111,191,32,"0,6","0,279","0,149",2209,Industrial and Manufacturing Engineering,23,259,336,Polish Society of Food Technologists,j,NO,4,FALSO,https://www.scopus.com/sourceid/19700173167,14256959,NaN


# Nueva sección